In [1]:
# an updated version of the uncertainty code that uses the different distribution 
# actually using keras models and trying to match the use of custom layers and losses.

In [1]:
#import necessary libraries
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as nd
from tensorflow import keras
%matplotlib inline
import pylab as pl
from IPython import display

# from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Download MNIST dataset
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
# print(y_train)
K= 10 # number of classes

In [3]:
# digit_one = mnist.train.images[4].copy()
# y_train = (y_train.map(tf.keras.to_categorical()))
# print(type(train_labels[0]))
# if type(train_labels[0]) is not type(np.ndarray(None,None)):
train_labels = tf.keras.utils.to_categorical(train_labels,10)
# print(type(np.ndarray(None,None)))
# if type(test_labels[0]) is not type(np.ndarray(None,None)):
test_labels = tf.keras.utils.to_categorical(test_labels,10)

    
print(train_labels[0])
print((test_labels[0]))
# plt.imshow(train_images[1].reshape(228,228)) 
# plt.show()

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [33]:
validation_size = 1000
shuffle_size = 1000
batch_size=32
validation_images, validation_labels = train_images[:validation_size], train_labels[:validation_size] #get the first 5k training samples as validation set
train_images, train_labels = train_images[validation_size:], train_labels[validation_size:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))


def augment_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    # image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label
train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))
train_ds = (train_ds.map(augment_images))
validation_ds = (validation_ds.map(augment_images))
test_ds = (test_ds.map(augment_images))

target = tf.data.Dataset.from_tensor_slices((train_labels))
train_ds = tf.data.Dataset.zip((train_ds,target))

v_target = tf.data.Dataset.from_tensor_slices((validation_labels))
validation_ds = tf.data.Dataset.zip((validation_ds,v_target))

t_target = tf.data.Dataset.from_tensor_slices((test_labels))
test_ds = tf.data.Dataset.zip((test_ds,t_target))


print("trainSize {}".format(train_ds_size))
print("testSize {}".format(test_ds_size))
train_ds = (train_ds
                
                .shuffle(buffer_size=tf.cast(shuffle_size,'int64'))
                .batch(batch_size=batch_size, drop_remainder=True))

test_ds = (test_ds
               
                #   .shuffle(buffer_size=train_ds_size)
                .batch(batch_size=1, drop_remainder=True))

validation_ds = (validation_ds
               
                #   .shuffle(buffer_size=validation_ds_size)
                .batch(batch_size=batch_size, drop_remainder=True))

trainSize 48000
testSize 10000


In [6]:
# Create a LeNet network with softmax cross entropy loss function
def LeNet_softmax(lmb=0.005):
    inputs = keras.Input(shape=(227,227,3))
    x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(4096, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(4096, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(10, activation='softmax')(x)
    
    # model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
    model = keras.Model(inputs=inputs, outputs=[x], name="alexnet")

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
#     model.summary()
    return model
model = LeNet_softmax()

In [ ]:
model.fit(train_ds,
          epochs=3,
          validation_data=validation_ds,
          validation_freq=1)
          


model.evaluate(test_ds)


Epoch 1/3
1468/1468 [==============================] - 98s 65ms/step - loss: 0.7524 - accuracy: 0.7364 - val_loss: 0.8411 - val_accuracy: 0.7107
Epoch 2/3
1468/1468 [==============================] - 105s 71ms/step - loss: 0.6783 - accuracy: 0.7624 - val_loss: 0.6536 - val_accuracy: 0.7792
Epoch 3/3
312/312 [==============================] - 5s 15ms/step - loss: 0.6853 - accuracy: 0.7647


[0.6852789521217346, 0.764723539352417]

In [5]:
# This function to generate evidence is used for the first example
def relu_evidence(logits):
    return tf.nn.relu(logits)

# This one usually works better and used for the second and third examples
# For general settings and different datasets, you may try this one first
def exp_evidence(logits): 
    return tf.exp(tf.clip_by_value(logits,-10,10))

# This one is another alternative and 
# usually behaves better than the relu_evidence 
def softplus_evidence(logits):
    return tf.nn.softplus(logits)

In [6]:
def KL(alpha):
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    S_beta = tf.reduce_sum(beta,axis=1,keepdims=True)
    lnB = tf.compat.v1.lgamma(S_alpha) - tf.reduce_sum(tf.compat.v1.lgamma(alpha),axis=1,keepdims=True)
    lnB_uni = tf.reduce_sum(tf.compat.v1.lgamma(beta),axis=1,keepdims=True) - tf.compat.v1.lgamma(S_beta)
    
    dg0 = tf.compat.v1.digamma(S_alpha)
    dg1 = tf.compat.v1.digamma(alpha)
    
    kl = tf.reduce_sum((alpha - beta)*(dg1-dg0),axis=1,keepdims=True) + lnB + lnB_uni
    return kl

def mse_loss(p, alpha, global_step, annealing_step): 
    S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
    E = alpha - 1
    m = alpha / S
    
    A = tf.reduce_sum((p-m)**2, axis=1, keepdims=True) 
    B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True) 

    annealing_coef = tf.minimum(1.0,tf.cast(global_step/annealing_step,tf.float32))
    
    alp = E*(1-p) + 1 
    C =  annealing_coef * KL(alp)
    return (A + B) + C

def loss_function():
    #create a wrapper function that returns a function
    kl = tf.keras.losses.KLDivergence()
    temperature = 1
    Classes = 10
    crossE = tf.keras.losses.CategoricalCrossentropy()
    mse =mse_loss
    global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
    annealing_step = 320
    def EMSE_Loss(y_true, y_pred):
#         softmax = tf.nn.softmax(y_pred)
        evidence = exp_evidence(y_pred)
        alpha  = evidence + 1
        
        u = Classes / tf.reduce_sum(alpha, axis=1) #uncertainty
        
        prob = alpha/tf.reduce_sum(alpha, 1,keepdims=True) 
        
        loss = tf.reduce_mean(mse(y_true, alpha,global_step,annealing_step))
#         l2_loss = (tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)) * lmb
#         l2_loss = tf.reduce_sum(tf.square(inputs - teaching_features))
        
        final_loss = loss #+ l2_loss
        
#         kl_loss = kl( tf.nn.softmax(softmax / self.temperature, axis = 1 ),
        
        return evidence, final_loss
    return  EMSE_Loss

def loss_function2():
#create a wrapper function that returns a function
    kl = tf.keras.losses.KLDivergence()
    temperature = 1
    Classes = 10
    crossE = tf.keras.losses.CategoricalCrossentropy()
    mse =tf.keras.losses.MeanSquaredError()
    def EMSE_Loss(y_true, y_pred):
#         softmax = tf.nn.softmax(y_pred)
        evidence = exp_evidence(y_pred)
        alpha  = evidence + 1
        u = Classes / tf.reduce_sum(alpha, axis=1) #uncertainty
        
        prob = alpha/tf.reduce_sum(alpha, 1,keepdims=True) 
        
        loss = tf.reduce_mean(mse(y_true, alpha))
#         l2_loss = (tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)) * lmb
#         l2_loss = tf.reduce_sum(tf.square(inputs - teaching_features))
        
        final_loss = loss #+ l2_loss
        
#         kl_loss = kl( tf.nn.softmax(softmax / self.temperature, axis = 1 ),
        
        return final_loss
    return  EMSE_Loss
    
    # return entropyAddition


In [29]:
class EvidenceEndpoint(keras.layers.Layer):
        def __init__(self, num_outputs, name=None, **kwargs):
            super(EvidenceEndpoint, self).__init__(name=name)
            self.num_outputs = num_outputs
            self.loss_coefficient = 1
            self.feature_loss_coefficient = 1
            self.kl = tf.keras.losses.KLDivergence()
            self.loss_fn = loss_function()
            self.temperature = 10
            self.alpha = .1
            self.lmb = 0.005
        def build(self, input_shape):
            self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]),
                                                    self.num_outputs])

        def call(self, inputs, labels):
            outputs = tf.matmul(inputs,self.kernel)
            # Compute the training-time loss value and add it
            # to the layer using `self.add_loss()`.
            #loss functions are (True, Prediction)
            # softmax = tf.nn.softmax(inputs)
            # print("inputs",input)
            #loss 1. normal loss, predictions vs labels
            evidence, normal_loss = self.loss_fn(labels, outputs)
            l2_loss = tf.nn.l2_loss(self.weights) * self.lmb
            # print(evidence.shape)
            # print(normal_loss)
            total_loss =tf.reduce_mean(normal_loss) + l2_loss
            # print(total_loss)
            total_evidence = tf.reduce_sum(evidence,1, keepdims=True) 
            # print(total_evidence)
            pred = tf.argmax(outputs, 1)
            truth = tf.argmax(labels, 1)
            match = tf.reshape(tf.cast(tf.equal(pred, truth), tf.float32),(-1,1))
            # print("match",match)
            mean_avg = tf.reduce_mean(total_evidence)
            # print("mean_Avg")
            
            mean_succ = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*match) / tf.reduce_sum(match+1e-20)
            # print("mean_fail")
            # side_x = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match))
            # print("side_x",side_x)
            # side_y = (tf.reduce_sum(tf.abs(1-match))+1e-20)
            # print("side_y",side_y)
            # div = tf.reduce_sum(side_x/side_y)
            # print("div",div)
            mean_fail = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match)) / (tf.reduce_sum(tf.abs(1-match))+1e-20) )
            # print('end')
            self.add_metric(normal_loss, name="normal_loss")
            self.add_metric(evidence, name="evidence")
            self.add_metric(mean_avg, name="mean_ev_avg")
            self.add_metric(mean_succ, name="mean_ev_succ")
            self.add_metric(mean_fail, name="mean_ev_fail")
            # print('metrics')
            self.add_loss(total_loss)
            # print("pred",inputs)
            # print("loss")
            #NOTE
            # The total loss is different from parts_loss because it includes the regularization term.
            # In other words, loss is computed as loss = parts_loss + k*R, where R is the regularization term 
            # (typically the L1 or L2 norm of the model's weights) and k a hyperparameter that controls the 
            # contribution of the regularization loss in the total loss.
            return outputs
    

In [31]:
# Create a LeNet network with softmax cross entropy loss function
def LeNet_softmax2(lmb=0.005):
    inputs = keras.Input(shape=(227,227,3))
    targets = keras.Input(shape=(10,),name="targets")
    x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(4096, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(4096, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    # x = keras.layers.Dense(10, activation='relu')(x)
    x = EvidenceEndpoint(10, activation='relu')(x,targets)
    
    # model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
    model = keras.Model(inputs=[inputs,targets], outputs=[x], name="alexnet")
    EMSE_loss = loss_function2()
    model.compile(  optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'],run_eagerly=True)
#     model.summary()
    return model
model = LeNet_softmax2()

In [32]:
model.fit(train_ds,
          epochs=3,
          validation_data=validation_ds,
          validation_freq=1)
          


model.evaluate(test_ds)

Epoch 1/3
1531/1531 [==============================] - 133s 86ms/step - loss: 1.0175 - accuracy: 0.1851 - normal_loss: 0.9203 - evidence: 10.1863 - mean_ev_avg: 101.8628 - mean_ev_succ: 258.2596 - mean_ev_fail: 40.1431 - val_loss: 0.8327 - val_accuracy: 0.3952 - val_normal_loss: 0.7907 - val_evidence: 17.4582 - val_mean_ev_avg: 174.5822 - val_mean_ev_succ: 405.9762 - val_mean_ev_fail: 22.1095
Epoch 2/3
1531/1531 [==============================] - 132s 86ms/step - loss: 0.8319 - accuracy: 0.4065 - normal_loss: 0.7558 - evidence: 36.8438 - mean_ev_avg: 368.4377 - mean_ev_succ: 718.4234 - mean_ev_fail: 103.4453 - val_loss: 0.6825 - val_accuracy: 0.5171 - val_normal_loss: 0.6477 - val_evidence: 39.5341 - val_mean_ev_avg: 395.3411 - val_mean_ev_succ: 689.6007 - val_mean_ev_fail: 79.8005
Epoch 3/3
312/312 [==============================] - 14s 43ms/step - loss: 0.5874 - accuracy: 0.5814 - normal_loss: 0.5577 - evidence: 96.1497 - mean_ev_avg: 961.4969 - mean_ev_succ: 1445.7648 - mean_ev_fail

[0.5874406099319458,
 0.5814303159713745,
 0.5577133297920227,
 96.14968872070312,
 961.4968872070312,
 1445.7647705078125,
 277.3133544921875]

In [34]:
model.evaluate(test_ds)

  848/10000 [=>............................] - ETA: 3:27 - loss: 0.6040 - accuracy: 0.5625 - normal_loss: 0.5742 - evidence: 91.3220 - mean_ev_avg: 913.2200 - mean_ev_succ: 800.7679 - mean_ev_fail: 112.4521

KeyboardInterrupt: 

In [ ]:
def draw_EDL_results(train_acc1, train_ev_s, train_ev_f, test_acc1, test_ev_s, test_ev_f): 
    # calculate uncertainty for training and testing data for correctly and misclassified samples
    train_u_succ = K / (K+np.array(train_ev_s))
    train_u_fail = K / (K+np.array(train_ev_f))
    test_u_succ  = K / (K+np.array(test_ev_s))
    test_u_fail  = K / (K+np.array(test_ev_f))
    
    f, axs = pl.subplots(2, 2)
    f.set_size_inches([10,10])
    
    axs[0,0].plot(train_ev_s,c='r',marker='+')
    axs[0,0].plot(train_ev_f,c='k',marker='x')
    axs[0,0].set_title('Train Data')
    axs[0,0].set_xlabel('Epoch')
    axs[0,0].set_ylabel('Estimated total evidence for classification') 
    axs[0,0].legend(['Correct Clasifications','Misclasifications'])
    
    
    axs[0,1].plot(train_u_succ,c='r',marker='+')
    axs[0,1].plot(train_u_fail,c='k',marker='x')
    axs[0,1].plot(train_acc1,c='blue',marker='*')
    axs[0,1].set_title('Train Data')
    axs[0,1].set_xlabel('Epoch')
    axs[0,1].set_ylabel('Estimated uncertainty for classification')
    axs[0,1].legend(['Correct clasifications','Misclasifications', 'Accuracy'])
    
    axs[1,0].plot(test_ev_s,c='r',marker='+')
    axs[1,0].plot(test_ev_f,c='k',marker='x')
    axs[1,0].set_title('Test Data')
    axs[1,0].set_xlabel('Epoch')
    axs[1,0].set_ylabel('Estimated total evidence for classification') 
    axs[1,0].legend(['Correct Clasifications','Misclasifications'])
    
    
    axs[1,1].plot(test_u_succ,c='r',marker='+')
    axs[1,1].plot(test_u_fail,c='k',marker='x')
    axs[1,1].plot(test_acc1,c='blue',marker='*')
    axs[1,1].set_title('Test Data')
    axs[1,1].set_xlabel('Epoch')
    axs[1,1].set_ylabel('Estimated uncertainty for classification')
    axs[1,1].legend(['Correct clasifications','Misclasifications', 'Accuracy'])
    
    plt.show()